In [6]:
#Lets import packages and define variables
from google.cloud import datacatalog_v1

project_id: str = "your project"
location_id: str = "your location"

In [8]:
#create a single client for your program
client = datacatalog_v1.PolicyTagManagerClient()

# Construct a full location path to be the parent of the taxonomy.
parent = datacatalog_v1.PolicyTagManagerClient.common_location_path(project_id, location_id)

Lets create Taxonomy pii-sensitive-taxonomy

In [ ]:
# Construct a full Taxonomy object to send to the API.
taxonomy = datacatalog_v1.Taxonomy(
    display_name = 'pii-sensitive-taxonomy',
    description = "This Taxonomy represents ...")

# Send the taxonomy to the API for creation.
taxonomy = client.create_taxonomy(parent=parent, taxonomy=taxonomy)
print(f"Created taxonomy {taxonomy.display_name} : {taxonomy.name}")

Next we will create a first policy tag under pii-sensitive-taxonomy

In [ ]:
#construct Catagory tag named "high" that is the severity category high
policyTag = datacatalog_v1.types.PolicyTag( 
    name = "high",
    display_name = "high",
    description = 'Severity Category high')

request = datacatalog_v1.CreatePolicyTagRequest(
    parent=taxonomy.name,
    policy_tag=policyTag)

#create policy tag using request created above
response = client.create_policy_tag(request=request)

print(f"Tag Created {response.display_name}: {response.name}")

Create subtags in high category

In [ ]:

#Use response of first tag to define "high" as a parent policy tag to create a subtag
parent_tag_name = response.name

#construct child tag
child_tag = datacatalog_v1.PolicyTag(
        name = "driving_license",
        display_name="driving_license",
        parent_policy_tag=parent_tag_name
    )

#define request
request = datacatalog_v1.CreatePolicyTagRequest(
        parent=taxonomy.name,
        policy_tag=child_tag
    )
  
response = client.create_policy_tag(request=request)

print(f"Subtag created :{response.name}")

Lets creare data masking policy

In [ ]:
#Lets import packages and define variables for data masking rules
from google.cloud import bigquery
from google.cloud import bigquery_datapolicies_v1
from google.cloud.bigquery_datapolicies_v1 import DataPolicyServiceClient
from google.cloud.bigquery_datapolicies_v1.types import DataPolicy, DataMaskingPolicy

#Lets create a Client and parent for request
client = bigquery_datapolicies_v1.DataPolicyServiceClient()

parent = f"projects//{project_id}locations/us"

In [ ]:
# Initialize request argument(s) for our data policy for Sales team who just need to see last 4 chars of driving lisence
data_policy = bigquery_datapolicies_v1.DataPolicy()
data_policy.name="pii_last_four"
data_policy.data_policy_id="pii_last_four"
data_policy.data_policy_type = "DATA_MASKING_POLICY"

#Below use the policy tag id for child tag that we creadted above for driving_license
data_policy.policy_tag = "projects/{project_id}/locations/us/taxonomies/{taxonomy_id}/policyTags/{policyTagid}"
data_policy.data_masking_policy.predefined_expression = "LAST_FOUR_CHARACTERS"

request = bigquery_datapolicies_v1.CreateDataPolicyRequest(
        parent=parent,
        data_policy=data_policy,
    )

# Make the request
response = client.create_data_policy(request=request)

# Handle the response
print(response)

In [ ]:
# Initialize request argument(s) for our data policy for Analysts team who do not need to see the original value but just the hash value
data_policy = bigquery_datapolicies_v1.DataPolicy()
data_policy.name="pii_hash"
data_policy.data_policy_id="pii_hash"
data_policy.data_policy_type = "DATA_MASKING_POLICY"
data_policy.policy_tag = "projects/{project_id}/locations/us/taxonomies/{taxonomy_id}/policyTags/{policyTagid}"
data_policy.data_masking_policy.predefined_expression = "SHA256"

request = bigquery_datapolicies_v1.CreateDataPolicyRequest(
        parent=parent,
        data_policy=data_policy,
    )

# Make the request
response = client.create_data_policy(request=request)

# Handle the response
print(response)